In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

df = pd.read_csv('datasets/Social_Network_Ads.csv')

# transformar coluna com variável categórica em variável binária (dummy)
df['Gender'] = df.apply(lambda row: 1 if row['Gender']=='Female' else 0, axis=1)

# df.head()
# df.count()
# Dividir minha base total em teste e treino
df_train = df.sample(200)

# retiro da base principal as observações que utilizo na base de treino
df = df.drop(df_train.index)

# Crio uma lista com 10 amostras de 50 observações cada a partir
# das 200 observações que sobraram após a retirada das observações
# de treino. HAVERÁ REPETIÇÃO
samples = []
for i in range(10):
  samples.append(df.sample(50))


In [3]:
# Criei minhas variáveis de treino
X_train = df_train[['Gender','Age','EstimatedSalary']].to_numpy()
Y_train = df_train['Purchased'].to_numpy()

# Criei meu modelo de Regressão Linear
model_linear = LinearRegression()
model_linear.fit(X_train, Y_train)
score_linear = model_linear.score(X_train, Y_train)

# Criei meu modelo de Regressão Polinomial
poly_features = PolynomialFeatures(degree=3)
X_poly_train = poly_features.fit_transform(X_train)
model_poly = LinearRegression()
model_poly.fit(X_poly_train, Y_train)
score_polynomial = model_poly.score(X_poly_train, Y_train)

# Começar a testar o modelo
test_score_linear = []
test_score_polynomial = []
for sample in samples:
  # Criando variáveis X, Y para regressão linear
  X_test = sample[['Gender','Age','EstimatedSalary']].to_numpy()
  Y_test = sample['Purchased'].to_numpy()
  test_score_linear.append(model_linear.score(X_test, Y_test))

  # Criando variável polinomial para o X
  X_test_poly = poly_features.fit_transform(X_test)
  test_score_polynomial.append(model_poly.score(X_test_poly, Y_test))

alpha = 0.1
# Modelagem do teste de hipóteses
# H0: score de teste é igual ao score de treino
#     a minha regressão não perde poder de explicação quando testa novas variáveis
# H1: score de teste é diferente do score de treino
#     a minha regressão não explica a variável de interesse da mesma maneira que o teste.

t_stat, p_valor = stats.ttest_1samp(test_score_linear, popmean=score_linear)
print(score_linear)
print(p_valor)
if p_valor < alpha:
  print("Regressão Linear: Aceito H1 e rejeito H0")
else:
  print("Regressão Linear: Aceito H0 e rejeito H1")

t_stat, p_valor = stats.ttest_1samp(test_score_polynomial, popmean=score_polynomial)
print(score_polynomial)
print(p_valor)
if p_valor < alpha:
  print("Regressão Polinomial: Aceito H1 e rejeito H0")
else:
  print("Regressão Polinomial: Aceito H0 e rejeito H1")


0.47307010479218636
0.12490623270725018
Regressão Linear: Aceito H0 e rejeito H1
0.6579327242224804
0.01583242011785663
Regressão Polinomial: Aceito H1 e rejeito H0




Regressão linear:
y = beta0 + beta1 * x1

Regressão polinomial: grau 2
y = beta0 + beta1^2 * x1 + beta1^1 * 1

Regressão polinomial: grau 3
y = beta0 +  beta1^3 * x1 + beta1^2 * x1 + beta1^1 * 1



